In [1]:
import time
import random
import cv2
import os
import math
import numpy as np
from skimage.util import random_noise
from skimage import exposure


In [2]:
# -*- coding=utf-8 -*-
##############################################################
# description:
#     data augmentation for obeject detection
# author:
#     maozezhong 2018-6-27
##############################################################

# 包括:
#     1. 裁剪(需改变bbox)
#     2. 平移(需改变bbox)
#     3. 改变亮度
#     4. 加噪声
#     5. 旋转角度(需要改变bbox)
#     6. 镜像(需要改变bbox)
#     7. cutout
# 注意:   
#     random.seed(),相同的seed,产生的随机数是一样的!!

def show_pic(img, bboxes=None):
    '''
    输入:
        img:图像array
        bboxes:图像的所有boudning box list, 格式为[[x_min, y_min, x_max, y_max]....]
        names:每个box对应的名称
    '''
    cv2.imwrite('./1.jpg', img)
    img = cv2.imread('./1.jpg')
    for i in range(len(bboxes)):
        bbox = bboxes[i]
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[2]
        y_max = bbox[3]
        cv2.rectangle(img,(int(x_min),int(y_min)),(int(x_max),int(y_max)),(0,255,0),3) 
    cv2.namedWindow('pic', 0)  # 1表示原图
    cv2.moveWindow('pic', 0, 0)
    cv2.resizeWindow('pic', 1200,800)  # 可视化的图片大小
    cv2.imshow('pic', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows() 
    os.remove('./1.jpg')
	
def save_aug_img(img,img_path_name):
    cv2.imwrite(img_path_name, img)
	
# 图像均为cv2读取
class DataAugmentForObjectDetection():
    def __init__(self, rotation_rate=0.5, max_rotation_angle=5, 
                crop_rate=0.5, shift_rate=0.5, change_light_rate=0.5,
                add_noise_rate=0.5, flip_rate=0.5, 
                cutout_rate=0.5, cut_out_length=50, cut_out_holes=1, cut_out_threshold=0.5):
        self.rotation_rate = rotation_rate
        self.max_rotation_angle = max_rotation_angle
        self.crop_rate = crop_rate
        self.shift_rate = shift_rate
        self.change_light_rate = change_light_rate
        self.add_noise_rate = add_noise_rate
        self.flip_rate = flip_rate
        self.cutout_rate = cutout_rate

        self.cut_out_length = cut_out_length
        self.cut_out_holes = cut_out_holes
        self.cut_out_threshold = cut_out_threshold
    
    # 加噪声
    def _addNoise(self, img):
        '''
        输入:
            img:图像array
        输出:
            加噪声后的图像array,由于输出的像素是在[0,1]之间,所以得乘以255
        '''
        # random.seed(int(time.time())) 
        # return random_noise(img, mode='gaussian', seed=int(time.time()), clip=True)*255
        return random_noise(img, mode='gaussian', clip=True)*255

    
    # 调整亮度
    def _changeLight(self, img):
        # random.seed(int(time.time()))
        flag = random.uniform(0.5, 1.5) #flag>1为调暗,小于1为调亮
        return exposure.adjust_gamma(img, flag)
    
    # cutout
    def _cutout(self, img, bboxes, length=100, n_holes=1, threshold=0.5):
        '''
        原版本：https://github.com/uoguelph-mlrg/Cutout/blob/master/util/cutout.py
        Randomly mask out one or more patches from an image.
        Args:
            img : a 3D numpy array,(h,w,c)
            bboxes : 框的坐标
            n_holes (int): Number of patches to cut out of each image.
            length (int): The length (in pixels) of each square patch.
        '''
        
        def cal_iou(boxA, boxB):
            '''
            boxA, boxB为两个框，返回iou
            boxB为bouding box
            '''

            # determine the (x, y)-coordinates of the intersection rectangle
            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])

            if xB <= xA or yB <= yA:
                return 0.0

            # compute the area of intersection rectangle
            interArea = (xB - xA + 1) * (yB - yA + 1)

            # compute the area of both the prediction and ground-truth
            # rectangles
            boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
            boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

            # compute the intersection over union by taking the intersection
            # area and dividing it by the sum of prediction + ground-truth
            # areas - the interesection area
            # iou = interArea / float(boxAArea + boxBArea - interArea)
            iou = interArea / float(boxBArea)

            # return the intersection over union value
            return iou

        # 得到h和w
        if img.ndim == 3:
            h,w,c = img.shape
        else:
            _,h,w,c = img.shape
        
        mask = np.ones((h,w,c), np.float32)

        for n in range(n_holes):
            
            chongdie = True    #看切割的区域是否与box重叠太多
            
            while chongdie:
                y = np.random.randint(h)
                x = np.random.randint(w)

                y1 = np.clip(y - length // 2, 0, h)    #numpy.clip(a, a_min, a_max, out=None), clip这个函数将将数组中的元素限制在a_min, a_max之间，大于a_max的就使得它等于 a_max，小于a_min,的就使得它等于a_min
                y2 = np.clip(y + length // 2, 0, h)
                x1 = np.clip(x - length // 2, 0, w)
                x2 = np.clip(x + length // 2, 0, w)

                chongdie = False
                for box in bboxes:
                    if cal_iou([x1,y1,x2,y2], box) > threshold:
                        chongdie = True
                        break
            
            mask[y1: y2, x1: x2, :] = 0.
        
        # mask = np.expand_dims(mask, axis=0)
        img = img * mask

        return img

    # 旋转
    def _rotate_img_bbox(self, img, bboxes, angle=5, scale=1.):
        '''
        参考:https://blog.csdn.net/u014540717/article/details/53301195crop_rate
        输入:
            img:图像array,(h,w,c)
            bboxes:该图像包含的所有boundingboxs,一个list,每个元素为[x_min, y_min, x_max, y_max],要确保是数值
            angle:旋转角度
            scale:默认1
        输出:
            rot_img:旋转后的图像array
            rot_bboxes:旋转后的boundingbox坐标list
        '''
        #---------------------- 旋转图像 ----------------------
        w = img.shape[1]
        h = img.shape[0]
        # 角度变弧度
        rangle = np.deg2rad(angle)  # angle in radians
        # now calculate new image width and height
        nw = (abs(np.sin(rangle)*h) + abs(np.cos(rangle)*w))*scale
        nh = (abs(np.cos(rangle)*h) + abs(np.sin(rangle)*w))*scale
        # ask OpenCV for the rotation matrix
        rot_mat = cv2.getRotationMatrix2D((nw*0.5, nh*0.5), angle, scale)
        # calculate the move from the old center to the new center combined
        # with the rotation
        rot_move = np.dot(rot_mat, np.array([(nw-w)*0.5, (nh-h)*0.5,0]))
        # the move only affects the translation, so update the translation
        # part of the transform
        rot_mat[0,2] += rot_move[0]
        rot_mat[1,2] += rot_move[1]
        # 仿射变换
        rot_img = cv2.warpAffine(img, rot_mat, (int(math.ceil(nw)), int(math.ceil(nh))), flags=cv2.INTER_LANCZOS4)

        #---------------------- 矫正bbox坐标 ----------------------
        # rot_mat是最终的旋转矩阵
        # 获取原始bbox的四个中点，然后将这四个点转换到旋转后的坐标系下
        rot_bboxes = list()
        for bbox in bboxes:
            xmin = bbox[0]
            ymin = bbox[1]
            xmax = bbox[2]
            ymax = bbox[3]
            point1 = np.dot(rot_mat, np.array([(xmin+xmax)/2, ymin, 1]))
            point2 = np.dot(rot_mat, np.array([xmax, (ymin+ymax)/2, 1]))
            point3 = np.dot(rot_mat, np.array([(xmin+xmax)/2, ymax, 1]))
            point4 = np.dot(rot_mat, np.array([xmin, (ymin+ymax)/2, 1]))
            # 合并np.array
            concat = np.vstack((point1, point2, point3, point4))
            # 改变array类型
            concat = concat.astype(np.int32)
            # 得到旋转后的坐标
            rx, ry, rw, rh = cv2.boundingRect(concat)
            rx_min = rx
            ry_min = ry
            rx_max = rx+rw
            ry_max = ry+rh
            # 加入list中
            rot_bboxes.append([rx_min, ry_min, rx_max, ry_max])
        
        return rot_img, rot_bboxes

    # 裁剪
    def _crop_img_bboxes(self, img, bboxes):
        '''
        裁剪后的图片要包含所有的框
        输入:
            img:图像array
            bboxes:该图像包含的所有boundingboxs,一个list,每个元素为[x_min, y_min, x_max, y_max],要确保是数值
        输出:
            crop_img:裁剪后的图像array
            crop_bboxes:裁剪后的bounding box的坐标list
        '''
        #---------------------- 裁剪图像 ----------------------
        w = img.shape[1]
        h = img.shape[0]
        x_min = w   #裁剪后的包含所有目标框的最小的框
        x_max = 0
        y_min = h
        y_max = 0
        for bbox in bboxes:
            x_min = min(x_min, bbox[0])
            y_min = min(y_min, bbox[1])
            x_max = max(x_max, bbox[2])
            y_max = max(y_max, bbox[3])
        
        d_to_left = x_min           #包含所有目标框的最小框到左边的距离
        d_to_right = w - x_max      #包含所有目标框的最小框到右边的距离
        d_to_top = y_min            #包含所有目标框的最小框到顶端的距离
        d_to_bottom = h - y_max     #包含所有目标框的最小框到底部的距离

        #随机扩展这个最小框
        crop_x_min = int(x_min - random.uniform(0, d_to_left))
        crop_y_min = int(y_min - random.uniform(0, d_to_top))
        crop_x_max = int(x_max + random.uniform(0, d_to_right))
        crop_y_max = int(y_max + random.uniform(0, d_to_bottom))

        # 随机扩展这个最小框 , 防止别裁的太小
        # crop_x_min = int(x_min - random.uniform(d_to_left//2, d_to_left))
        # crop_y_min = int(y_min - random.uniform(d_to_top//2, d_to_top))
        # crop_x_max = int(x_max + random.uniform(d_to_right//2, d_to_right))
        # crop_y_max = int(y_max + random.uniform(d_to_bottom//2, d_to_bottom))

        #确保不要越界
        crop_x_min = max(0, crop_x_min)
        crop_y_min = max(0, crop_y_min)
        crop_x_max = min(w, crop_x_max)
        crop_y_max = min(h, crop_y_max)

        crop_img = img[crop_y_min:crop_y_max, crop_x_min:crop_x_max]
        
        #---------------------- 裁剪boundingbox ----------------------
        #裁剪后的boundingbox坐标计算
        crop_bboxes = list()
        for bbox in bboxes:
            crop_bboxes.append([bbox[0]-crop_x_min, bbox[1]-crop_y_min, bbox[2]-crop_x_min, bbox[3]-crop_y_min])
        
        return crop_img, crop_bboxes
  
    # 平移
    def _shift_pic_bboxes(self, img, bboxes):
        '''
        参考:https://blog.csdn.net/sty945/article/details/79387054
        平移后的图片要包含所有的框
        输入:
            img:图像array
            bboxes:该图像包含的所有boundingboxs,一个list,每个元素为[x_min, y_min, x_max, y_max],要确保是数值
        输出:
            shift_img:平移后的图像array
            shift_bboxes:平移后的bounding box的坐标list
        '''
        #---------------------- 平移图像 ----------------------
        w = img.shape[1]
        h = img.shape[0]
        x_min = w   #裁剪后的包含所有目标框的最小的框
        x_max = 0
        y_min = h
        y_max = 0
        for bbox in bboxes:
            x_min = min(x_min, bbox[0])
            y_min = min(y_min, bbox[1])
            x_max = max(x_max, bbox[2])
            y_max = max(y_max, bbox[3])
        
        d_to_left = x_min           #包含所有目标框的最大左移动距离
        d_to_right = w - x_max      #包含所有目标框的最大右移动距离
        d_to_top = y_min            #包含所有目标框的最大上移动距离
        d_to_bottom = h - y_max     #包含所有目标框的最大下移动距离

        x = random.uniform(-(d_to_left-1) / 3, (d_to_right-1) / 3)
        y = random.uniform(-(d_to_top-1) / 3, (d_to_bottom-1) / 3)
        
        M = np.float32([[1, 0, x], [0, 1, y]])  #x为向左或右移动的像素值,正为向右负为向左; y为向上或者向下移动的像素值,正为向下负为向上
        shift_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

        #---------------------- 平移boundingbox ----------------------
        shift_bboxes = list()
        for bbox in bboxes:
            shift_bboxes.append([bbox[0]+x, bbox[1]+y, bbox[2]+x, bbox[3]+y])

        return shift_img, shift_bboxes

    # 镜像
    def _filp_pic_bboxes(self, img, bboxes):
        '''
            参考:https://blog.csdn.net/jningwei/article/details/78753607
            平移后的图片要包含所有的框
            输入:
                img:图像array
                bboxes:该图像包含的所有boundingboxs,一个list,每个元素为[x_min, y_min, x_max, y_max],要确保是数值
            输出:
                flip_img:平移后的图像array
                flip_bboxes:平移后的bounding box的坐标list
        '''
        # ---------------------- 翻转图像 ----------------------
        import copy
        flip_img = copy.deepcopy(img)
#         if random.random() < 0.5:    #0.5的概率水平翻转，0.5的概率垂直翻转
#             horizon = True
#         else:
#             horizon = False
        horizon = False
        h,w,_ = img.shape
        if horizon: #水平翻转
            flip_img =  cv2.flip(flip_img, -1)
        else:
            flip_img = cv2.flip(flip_img, 0)

        # ---------------------- 调整boundingbox ----------------------
        flip_bboxes = list()
        for box in bboxes:
            x_min = box[0]
            y_min = box[1]
            x_max = box[2]
            y_max = box[3]
            if horizon:
                flip_bboxes.append([w-x_max, y_min, w-x_min, y_max])
            else:
                flip_bboxes.append([x_min, h-y_max, x_max, h-y_min])

        return flip_img, flip_bboxes

    def dataAugment(self, img, bboxes):
        '''
        图像增强
        输入:
            img:图像array
            bboxes:该图像的所有框坐标
        输出:
            img:增强后的图像
            bboxes:增强后图片对应的box
        '''
        change_num = 0  #改变的次数
        print('------')
#         while change_num < 1:   #默认至少有一种数据增强生效
#             if random.random() < self.crop_rate:        #裁剪
#                 print('裁剪')
#                 change_num += 1
#                 img, bboxes = self._crop_img_bboxes(img, bboxes)
            
#             if random.random() > self.rotation_rate:    #旋转
#                 print('旋转')
#                 change_num += 1
#                 # angle = random.uniform(-self.max_rotation_angle, self.max_rotation_angle)
#                 angle = random.sample([90, 180, 270],1)[0]
#                 scale = random.uniform(0.7, 0.8)
#                 img, bboxes = self._rotate_img_bbox(img, bboxes, angle, scale)
            
#             if random.random() < self.shift_rate:        #平移
#                 print('平移')
#                 change_num += 1
#                 img, bboxes = self._shift_pic_bboxes(img, bboxes)
            
#             if random.random() > self.change_light_rate: #改变亮度
#                 print('亮度')
#                 change_num += 1
#                 img = self._changeLight(img)
            
#             if random.random() < self.add_noise_rate:    #加噪声
#                 print('加噪声')
#                 change_num += 1
#                 img = self._addNoise(img)

#             if random.random() < self.cutout_rate:  #cutout
#                 print('cutout')
#                 change_num += 1
#                 img = self._cutout(img, bboxes, length=self.cut_out_length, n_holes=self.cut_out_holes, threshold=self.cut_out_threshold)

#             if random.random() < self.flip_rate:    #翻转
#                 print('翻转')
#                 change_num += 1
#                 img, bboxes = self._filp_pic_bboxes(img, bboxes)
#             print('\n')
        # print('------')
        img, bboxes = self._filp_pic_bboxes(img, bboxes)
        return img, bboxes
            






In [3]:
import shutil
from xml_helper import *


need_aug_num = 1                  

dataAug = DataAugmentForObjectDetection()

source_pic_root_path = './JPEGImages'
source_xml_root_path = './Annotations'

    
for parent, _, files in os.walk(source_pic_root_path):
    for file in files:
        cnt = 0
#         while cnt < need_aug_num:
        pic_path = os.path.join(parent, file)
        xml_path = os.path.join(source_xml_root_path, file[:-4]+'.xml')
        coords_ = parse_xml(xml_path)        #解析得到box信息，格式为[[x_min,y_min,x_max,y_max,name]]
        coords = [coord[:4] for coord in coords_]
        
        img = cv2.imread(pic_path)
            #show_pic(img, coords)    # 原图

        auged_img, auged_bboxes = dataAug.dataAugment(img, coords)
        i = 0
        coords_aug = []
        box_ = []
        for box in auged_bboxes:
            box.append(coords_[i][-1])
            coords_aug.append(box)
            i += 1
#           cnt += 1
        print(coords_)
        print(coords_aug)
            #保存增强图片并生产xml文件
        img_name = '0%s'%(file[:-4])
        img_size = auged_img.shape
        save_aug_img(auged_img,"./imgout_hor_flip/%s.jpg"%img_name)
        generate_xml(img_name,coords_aug,
                        img_size ,"./xmlout_hor_flip")
            #show_pic(auged_img, auged_bboxes)  # 强化后的图

------
[[476, 1824, 675, 2130, '褐青色痣'], [674, 2047, 1085, 2222, '黄褐斑'], [1069, 1214, 1127, 1282, '炎症性色沉着'], [990, 1341, 1602, 1603, '炎症性色沉着'], [1544, 2272, 1592, 2335, '色素痣'], [1929, 1797, 1958, 1833, '色素痣'], [1748, 2018, 1963, 2306, '褐青色痣'], [1138, 2159, 1362, 2485, '黑头']]
[[476, 1134, 675, 1440, '褐青色痣'], [674, 1042, 1085, 1217, '黄褐斑'], [1069, 1982, 1127, 2050, '炎症性色沉着'], [990, 1661, 1602, 1923, '炎症性色沉着'], [1544, 929, 1592, 992, '色素痣'], [1929, 1431, 1958, 1467, '色素痣'], [1748, 958, 1963, 1246, '褐青色痣'], [1138, 779, 1362, 1105, '黑头']]
------
[[1919, 2589, 2019, 2721, '晒斑'], [723, 2502, 1426, 2817, '雀斑'], [1184, 3030, 1831, 3206, '雀斑'], [1323, 2580, 1750, 2929, '黑头']]
[[1919, 1311, 2019, 1443, '晒斑'], [723, 1215, 1426, 1530, '雀斑'], [1184, 826, 1831, 1002, '雀斑'], [1323, 1103, 1750, 1452, '黑头']]
------
[]
[]
------
[[1691, 2353, 1738, 2418, '色素痣'], [1758, 2418, 1805, 2472, '色素痣'], [389, 1751, 602, 2044, '黄褐斑'], [1824, 1813, 1921, 2124, '黄褐斑'], [1129, 1855, 1417, 2166, '雀斑']]
[[1691, 846, 173

------
[[1945, 2321, 2004, 2437, '痤疮'], [422, 2299, 753, 2572, '炎症性色沉着'], [1690, 2473, 1784, 2554, '痤疮']]
[[1945, 827, 2004, 943, '痤疮'], [422, 692, 753, 965, '炎症性色沉着'], [1690, 710, 1784, 791, '痤疮']]
------
[[1169, 1827, 1230, 1888, '色素痣'], [1238, 2157, 1303, 2233, '色素痣'], [1075, 2513, 1154, 2574, '色素痣'], [948, 2774, 1020, 2842, '色素痣'], [565, 1969, 1043, 2230, '雀斑'], [1490, 2020, 1965, 2270, '雀斑']]
[[1169, 1376, 1230, 1437, '色素痣'], [1238, 1031, 1303, 1107, '色素痣'], [1075, 690, 1154, 751, '色素痣'], [948, 422, 1020, 490, '色素痣'], [565, 1034, 1043, 1295, '雀斑'], [1490, 994, 1965, 1244, '雀斑']]
------
[]
[]
------
[[1604, 2074, 1801, 2360, '黄褐斑'], [901, 2156, 1212, 2491, '黑头'], [975, 3011, 1016, 3045, '色素痣']]
[[1604, 904, 1801, 1190, '黄褐斑'], [901, 773, 1212, 1108, '黑头'], [975, 219, 1016, 253, '色素痣']]
------
[[547, 2083, 688, 2315, '老年斑'], [1091, 1822, 1210, 2055, '老年斑'], [1269, 1965, 1865, 2293, '雀斑'], [748, 1978, 1050, 2353, '雀斑'], [1115, 1891, 1218, 2045, '晒斑'], [551, 2088, 697, 2298, '老年斑'], [

------
[[1673, 1187, 1745, 1266, '晒斑'], [1474, 1842, 1967, 2112, '黄褐斑'], [518, 1887, 702, 2184, '黄褐斑'], [641, 1465, 720, 1530, '晒斑'], [1318, 1800, 1423, 1948, '晒斑'], [1035, 1354, 1515, 1540, '炎症性色沉着']]
[[1673, 1998, 1745, 2077, '晒斑'], [1474, 1152, 1967, 1422, '黄褐斑'], [518, 1080, 702, 1377, '黄褐斑'], [641, 1734, 720, 1799, '晒斑'], [1318, 1316, 1423, 1464, '晒斑'], [1035, 1724, 1515, 1910, '炎症性色沉着']]
------
[[1612, 2596, 1677, 2672, '色素痣'], [1478, 2030, 1908, 2269, '雀斑'], [956, 2534, 1430, 2665, '雀斑']]
[[1612, 592, 1677, 668, '色素痣'], [1478, 995, 1908, 1234, '雀斑'], [956, 599, 1430, 730, '雀斑']]
------
[[721, 2573, 1258, 2873, '雀斑'], [1695, 2542, 2284, 2864, '雀斑'], [1355, 2329, 1605, 2700, '雀斑']]
[[721, 1159, 1258, 1459, '雀斑'], [1695, 1168, 2284, 1490, '雀斑'], [1355, 1332, 1605, 1703, '雀斑']]
------
[]
[]
------
[[1659, 1933, 1856, 2175, '褐青色痣'], [409, 1996, 639, 2260, '褐青色痣'], [993, 2596, 1361, 2726, '炎症性色沉着'], [585, 2328, 704, 2525, '炎症性色沉着']]
[[1659, 1089, 1856, 1331, '褐青色痣'], [409, 1004, 639, 

------
[[1561, 2618, 1695, 2725, '炎症性色沉着'], [1924, 2880, 1986, 2960, '色素痣'], [669, 2235, 1280, 2474, '雀斑'], [1841, 2269, 2342, 2515, '雀斑']]
[[1561, 1307, 1695, 1414, '炎症性色沉着'], [1924, 1072, 1986, 1152, '色素痣'], [669, 1558, 1280, 1797, '雀斑'], [1841, 1517, 2342, 1763, '雀斑']]
------
[[509, 2059, 732, 2291, '褐青色痣'], [1720, 1934, 1865, 2213, '褐青色痣'], [1659, 2114, 1695, 2162, '痤疮']]
[[509, 973, 732, 1205, '褐青色痣'], [1720, 1051, 1865, 1330, '褐青色痣'], [1659, 1102, 1695, 1150, '痤疮']]
------
[[893, 2047, 1177, 2375, '黑头'], [1595, 1911, 1910, 2251, '褐青色痣'], [340, 1873, 557, 2128, '褐青色痣']]
[[893, 889, 1177, 1217, '黑头'], [1595, 1013, 1910, 1353, '褐青色痣'], [340, 1136, 557, 1391, '褐青色痣']]
------
[[1454, 2437, 1522, 2513, '色素痣'], [1068, 2556, 1151, 2624, '色素痣'], [1042, 2894, 1103, 2962, '色素痣'], [569, 2045, 960, 2331, '雀斑'], [1057, 1994, 1329, 2287, '雀斑'], [1432, 2041, 1888, 2356, '雀斑']]
[[1454, 751, 1522, 827, '色素痣'], [1068, 640, 1151, 708, '色素痣'], [1042, 302, 1103, 370, '色素痣'], [569, 933, 960, 1219, '雀斑'

------
[[710, 1916, 800, 1979, '老年斑']]
[[710, 1285, 800, 1348, '老年斑']]
------
[[1258, 1699, 1337, 1795, '老年斑'], [912, 1842, 968, 1905, '老年斑'], [1684, 1802, 1738, 1872, '老年斑'], [942, 2426, 972, 2464, '色素痣'], [986, 2576, 1016, 2610, '色素痣'], [1260, 1708, 1361, 1804, '老年斑'], [1546, 1784, 1957, 2137, '黄褐斑'], [592, 1795, 1014, 2130, '黄褐斑'], [942, 2431, 980, 2463, '色素痣'], [984, 2573, 1020, 2607, '色素痣']]
[[1258, 1469, 1337, 1565, '老年斑'], [912, 1359, 968, 1422, '老年斑'], [1684, 1392, 1738, 1462, '老年斑'], [942, 800, 972, 838, '色素痣'], [986, 654, 1016, 688, '色素痣'], [1260, 1460, 1361, 1556, '老年斑'], [1546, 1127, 1957, 1480, '黄褐斑'], [592, 1134, 1014, 1469, '黄褐斑'], [942, 801, 980, 833, '色素痣'], [984, 657, 1020, 691, '色素痣']]
------
[[1044, 1865, 1323, 2080, '雀斑'], [1437, 1965, 1832, 2159, '雀斑'], [435, 2014, 955, 2267, '雀斑']]
[[1044, 1184, 1323, 1399, '雀斑'], [1437, 1105, 1832, 1299, '雀斑'], [435, 997, 955, 1250, '雀斑']]
------
[[571, 1708, 636, 1756, '色素痣'], [1597, 1777, 1635, 1816, '色素痣'], [1983, 2208, 2028,

...

------
[]
[]
------
[[946, 2076, 1203, 2299, '黑头'], [538, 1963, 670, 2059, '老年斑'], [986, 1934, 1096, 2010, '老年斑'], [1760, 1778, 1839, 1865, '老年斑'], [1644, 2032, 1680, 2071, '色素痣'], [1786, 1893, 1824, 1941, '色素痣'], [1086, 2408, 1160, 2464, '色素痣'], [603, 2215, 637, 2258, '色素痣'], [458, 2052, 492, 2097, '色素痣'], [997, 1416, 1040, 1468, '色素痣']]
[[946, 965, 1203, 1188, '黑头'], [538, 1205, 670, 1301, '老年斑'], [986, 1254, 1096, 1330, '老年斑'], [1760, 1399, 1839, 1486, '老年斑'], [1644, 1193, 1680, 1232, '色素痣'], [1786, 1323, 1824, 1371, '色素痣'], [1086, 800, 1160, 856, '色素痣'], [603, 1006, 637, 1049, '色素痣'], [458, 1167, 492, 1212, '色素痣'], [997, 1796, 1040, 1848, '色素痣']]
------
[[1488, 2009, 1964, 2270, '褐青色痣'], [518, 2016, 964, 2260, '褐青色痣'], [1423, 2420, 1457, 2467, '色素痣'], [1577, 2741, 1613, 2799, '色素痣'], [1550, 2854, 1580, 2899, '色素痣'], [1679, 1767, 1729, 1812, '色素痣']]
[[1488, 994, 1964, 1255, '褐青色痣'], [518, 1004, 964, 1248, '褐青色痣'], [1423, 797, 1457, 844, '色素痣'], [1577, 465, 1613, 523, '色素痣'], [1550, 

------
[[1088, 1428, 1200, 1529, '老年斑'], [638, 1998, 697, 2052, '老年斑'], [1285, 1960, 1794, 2371, '黄褐斑'], [377, 1925, 957, 2229, '黄褐斑']]
[[1088, 1735, 1200, 1836, '老年斑'], [638, 1212, 697, 1266, '老年斑'], [1285, 893, 1794, 1304, '黄褐斑'], [377, 1035, 957, 1339, '黄褐斑']]
------
[]
[]
------
[]
[]
------
[[361, 976, 699, 1176, '褐青色痣'], [1184, 894, 1447, 1169, '褐青色痣']]
[[361, 1416, 699, 1616, '褐青色痣'], [1184, 1423, 1447, 1698, '褐青色痣']]
------
[[754, 2307, 1056, 2656, '褐青色痣'], [1850, 2302, 2248, 2655, '褐青色痣'], [1379, 2553, 1482, 2660, '炎症性色沉着']]
[[754, 1376, 1056, 1725, '褐青色痣'], [1850, 1377, 2248, 1730, '褐青色痣'], [1379, 1372, 1482, 1479, '炎症性色沉着']]
------
[]
[]
------
[]
[]
------
[[1483, 2085, 1555, 2164, '老年斑'], [948, 1992, 1664, 2421, '黄褐斑'], [959, 2433, 1006, 2476, '色素痣'], [823, 2861, 864, 2911, '色素痣']]
[[1483, 1100, 1555, 1179, '老年斑'], [948, 843, 1664, 1272, '黄褐斑'], [959, 788, 1006, 831, '色素痣'], [823, 353, 864, 403, '色素痣']]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
-----

------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[[393, 1651, 588, 2100, '褐青色痣'], [1675, 2012, 2104, 2352, '褐青色痣'], [718, 2295, 852, 2440, '老年斑'], [716, 1350, 772, 1409, '褐青色痣']]
[[393, 1164, 588, 1613, '褐青色痣'], [1675, 912, 2104, 1252, '褐青色痣'], [718, 824, 852, 969, '老年斑'], [716, 1855, 772, 1914, '褐青色痣']]
------
[[357, 1571, 760, 1798, '褐青色痣'], [1033, 1573, 1498, 1797, '褐青色痣'], [800, 1203, 1024, 1442, '炎症性色沉着'], [728, 2295, 1079, 2413, '炎症性色沉着'], [1222, 1951, 1250, 1989, '色素痣']]
[[357, 794, 760, 1021, '褐青色痣'], [1033, 795, 1498, 1019, '褐青色痣'], [800, 1150, 1024, 1389, '炎症性色沉着'], [728, 179, 1079, 297, '炎症性色沉着'], [1222, 603, 1250, 641, '色素痣']]
------
[]
[]
------
[]
[]
------
[[605, 1784, 692, 1903, '老年斑'], [1477, 1913, 1719, 2331, '黄褐斑']]
[[605, 1361, 692, 1480, '老年斑'], [1477, 933, 1719, 1351, '黄褐斑']]
------
[]
[]
------
[]
[]
------
[]
[]
------
[[1505, 2571, 1648, 2745, '炎症性色沉着'], [1026, 1728, 1207, 1896, '炎症性色沉着'], [1691, 1958, 1885, 2306, '褐青色痣'], [525, 1996, 740, 2224, '褐青色痣

------
[]
[]
------
[[1648, 1425, 1751, 1526, '老年斑'], [1570, 2121, 1898, 2398, '黄褐斑'], [466, 2043, 783, 2378, '黄褐斑'], [759, 1445, 846, 1528, '老年斑']]
[[1648, 1738, 1751, 1839, '老年斑'], [1570, 866, 1898, 1143, '黄褐斑'], [466, 886, 783, 1221, '黄褐斑'], [759, 1736, 846, 1819, '老年斑']]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[[1091, 2012, 1199, 2167, '老年斑'], [730, 2420, 844, 2514, '老年斑'], [1301, 2009, 1533, 2183, '褐青色痣'], [538, 2023, 596, 2100, '色素痣'], [565, 2034, 1016, 2333, '黄褐斑']]
[[1091, 1097, 1199, 1252, '老年斑'], [730, 750, 844, 844, '老年斑'], [1301, 1081, 1533, 1255, '褐青色痣'], [538, 1164, 596, 1241, '色素痣'], [565, 931, 1016, 1230, '黄褐斑']]
------
[]
[]
------
[]
[]
------
[[1948, 2701, 2030, 2817, '老年斑'], [2349, 2354, 2434, 2571, '老年斑']]
[[1948, 1215, 2030, 1331, '老年斑'], [2349, 1461, 2434, 1678, '老年斑']]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[

------
[]
[]
------
[]
[]
------
[[1798, 2114, 1890, 2217, '老年斑'], [1100, 2234, 1344, 2515, '黑头'], [1064, 1340, 1132, 1414, '痤疮'], [554, 2250, 891, 2478, '炎症性色沉着'], [1657, 2315, 1811, 2664, '炎症性色沉着']]
[[1798, 1047, 1890, 1150, '老年斑'], [1100, 749, 1344, 1030, '黑头'], [1064, 1850, 1132, 1924, '痤疮'], [554, 786, 891, 1014, '炎症性色沉着'], [1657, 600, 1811, 949, '炎症性色沉着']]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[]
[]
------
[[1622, 2282, 1716, 2430, '老年斑'], [977, 2406, 1227, 2694, '黑头'], [1347, 2186, 1426, 2265, '老年斑'], [233, 2159, 871, 2519, '黄褐斑']]
[[1622, 834, 1716, 982, '老年斑'], [977, 570, 1227, 858, '黑头'], [1347, 999, 1426, 1078, '老年斑'], [233, 745, 871, 1105, '黄褐斑']]
------
[]
[]
------
[[1583, 2228, 1668, 2309, '老年斑'], [484, 2203, 975, 2558, '黄褐斑'], [1300, 2186, 1709, 2488, '黄褐斑']]
[[1583, 955, 1668, 1036, '老年斑'], [484, 706, 975, 1061, '黄褐斑'], [1300, 776, 1709, 1078, '黄